# Carga del dataset *Brain MRI Images for Brain Tumor Detection*

Antes de empezar con el entrenamiento de los diferentes modelos de redes neuronales convolucionales, debemos asegurarnos de que los datos presentan un formato adecuado para ser utilizados como *input* de liberías como ``Keras`` o ``scikit-learn``. En concreto, si el proceso de entrenamiento se va a llevar a cabo con redes neuronales convolucionales (CNN) el *input* de dichas redes debe adoptar la forma de una matriz con dimensiones constantes. Dicho de otra forma, el proceso que llevemos a cabo en este cuaderno debe convertir todas las imágenes del conjunto de datos en matrices de las mismas dimensiones. En lo que respecta a nuestro trabajo, convertiremos todos nuestros *inputs* en imágenes de 224x224 píxeles. 

El primer paso para proceder con estas transformaciones, consistirá en realizar la carga de estas imágenes. Para ello, haremos uso del módulo ``os`` para recorrer los directorios que las contienen, así como el módulo ``matplotlib.pyplot`` y su función ``imread`` para leerlas y convertirlas en *arrays* de tipo ``numpy``. Una vez finalizado el proceso, las listas ``no_images`` y ``yes_images`` contendrán los arrays ``numpy`` que representan imágenes sin tejido canceroso y con tejido canceroso respectivamente.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [4]:
img_yes = '../data/yes/'
img_no = '../data/no/'

yes_files = os.listdir(img_yes)
no_files = os.listdir(img_no)

yes_images, no_images = [], []
print('[!] Reading files with positive diagnostic...')
for file in yes_files:
    yes_images.append(plt.imread(f'{img_yes}{file}'))
print('[!] Positive files read!')

print('[!] Reading files with negative diagnostic...')
for file in no_files:
    no_images.append(plt.imread(f'{img_no}{file}'))
print('[!] Negative files read!')

all_images = yes_images + no_images
YES_SIZE, NO_SIZE = len(yes_images), len(no_images)
TOTAL_SIZE = len(all_images)

[!] Reading files with positive diagnostic...
[!] Positive files read!
[!] Reading files with negative diagnostic...
[!] Negative files read!


Una vez realizada la carga de las imágenes, comenzaremos el procesamiento de las mismas en diferentes etapas. Por una parte, debemos prestar atención a cómo la función ``imread`` realiza la lectura de las imágenes. Como se ha explicado previamente, esta función lee la imágen y la transforma en un *array* de tipo ``numpy``. Sin embargo, en función de la naturaleza de la imágen los *arrays* obtenidos pueden tener diferentes dimensiones o diferente tipo (``dtype``). En este caso, distinguimos tres tipos de imágen:

1. **Imágenes en escala de gris**: estas imágenes vienen representadas como arrays bidimensionales (matrices) cuyas dimensiones coinciden con las dimensiones de la imagen. Por ejemplo, si realizamos la carga de una imágen de 512x512 píxeles obtendremos una matriz de 512 filas y 512 columnas. Cada posición de este array será un número entero entre 0 y 255 que representa una tonalidad de gris. Este tipo de imágenes presenta la ventaja de que sus píxeles pueden representarse como enteros sin signo de 8 bits (``uint8`` en ``numpy``), por lo que disminuyen significativamente los requisitos de memoria.

2. **Imágenes en escala RGB**: estas imágenes vienen representadas como arrays tridimensionales. Una forma de entender estas imágenes sería como un conjunto de tres matrices, cada una representando un color, teniendo en cuenta además que cada matriz tiene las mismas dimensiones que la propia imágen. Por ejemplo, si se realiza la carga de una imágen RGB de 512x512 píxeles, obtendríamos un array de dimensiones ``(512, 512, 3)``. De nuevo, cada elemento de este array será un número entero entre 0 y 255, por lo que podrá representarse como un entero sin signo de 8 bits.

3. **Imágenes con formato ``.png``**: estas imágenes vienen representadas como arrays de cuatro dimensiones, es decir, son imágenes de tipo RGBA (las imágenes RGBA surgen de la combinación de imágenes RGB con un cuarto canal de tipo *alpha* que sirve para indicar el grado de opacidad de cada píxel). La principal diferencia con los formatos anteriores, es que cada elemento del *array* es un número real entre 0 y 1 y por tanto, se representan en memoria mediante números en coma flotante de 32 bits (``float32`` en ``numpy``).

Para nuestro trabajo, utilizaremos como *input* imágenes en escala de gris de dimensiones 224x224 píxeles. Por tanto, el primer paso para procesar las imágenes consistirá en transformarlas a escala de gris. Para ello, se implementaron las siguientes funciones:

- ``convert_to_int_matrix(img_list)``: esta función recibe como argumento una lista con imágenes en formato *array* y convierte a formato entero aquellas imágenes que presentan elementos en coma flotante. Como se ha explicado previamente, las imágenes en formato ``.png`` se representan mediante números reales entre 0 y 1. Para realizar dicha conversión, multiplicamos cada elemento por 255 (que es el entero más alto en escala de grises) y posteriormente forzamos la conversión a entero de todos los elementos mediante el método ``astype`` de ``numpy``, utilizando ``np.uint8`` para indicar la transformación a entero de 8 bits sin signo.

- ``imgs_to_grayscale(img_list)``: esta función recibe como argumento una lista con imágenes en formato *array* y las convierte a imágenes en escala de gris. Para ello, se hace uso de la función ``cvtColor`` del módulo ``cv2`` (OpenCV2 para Python) con el código ``cv2.COLOR_BGR2GRAY`` como código de conversión. De esta forma, las imágenes se convierten a *arrays* bidimensionales que representan la imágen convertida a escala de grises.

Al realizar estas llamadas en el orden expuesto, las imágenes pasarán a estar representadas en escala de gris (con todos sus elementos entre 0 y 255). Una vez realizadas estas primeras transformaciones, se puede proceder a la siguiente etapa del preprocesado que consistirá en la eliminación del ruido en las imágenes. La mayoría de imágenes de nuestro dataset objeto contienen espacios y áreas no deseadas (como anotaciones en las esquinas) que podrían provocar un rendimiento pobre de los clasificadores a entrenar. Por tanto, nuestro objetivo consistirá en la eliminación de estas regiones ruidosas utilizando algoritmos que nos permitan detectar el contorno de los cerebros. El procedimiento que proponemos es muy similar al que se expone en [este artículo](https://www.sciencedirect.com/science/article/pii/S2666827020300049?via%3Dihub) y se explica en más detalle a continuación:

1. Conversión a imágenes binarias mediante *thresholding*: en primer lugar, se procedió con la binarización de las imágenes mediante la función  ``threshold`` del módulo ``cv2``. Esta operación consiste en seleccionar un valor de *threshold* y aplicarlo a cada píxel de la imágen. Si el píxel es menor que el valor de *threshold* su valor se fija a 0, en otro caso, su valor se fija a cierto valor máximo. En nuestro caso, fijamos a 30 el valor de threshold (``THRESH``) y a 255 el valor máximo (``MAX_VALUE``, que es el valor más alto que puede aparecer en la codificación de una imágen). De esta forma, lograremos una imágen en "blanco y negro", lo que facilitará el trabajo de los algoritmos de detección de contornos. 

2. Aplicación de operaciones de erosión y dilatación:

3. Difuminado *gaussiano*: 

4. Detección de contornos y puntos extremos:

In [3]:
THRESH = 30
MAX_VALUE = 255

def convert_to_int_matrix(img_list):
    MIN, MAX = 0, 255
    for i, img in enumerate(img_list):
        if img.dtype == np.float32:
            tmp = np.copy(img) * MAX
            img_list[i] = tmp.astype(np.uint8)

def resize_imgs(img_list, height, width):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.resize(img, (height, width))

def imgs_to_grayscale(img_list):
    for i, img in enumerate(img_list):
        if len(img.shape) == 3:
            img_list[i] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.GaussianBlur(img, (5, 5), 0) 

def threshold_imgs(img_list):
    for i, img in enumerate(img_list):
        _, img_list[i] = cv2.threshold(img, THRESH, MAX_VALUE, cv2.THRESH_BINARY)

def erode_imgs(img_list, iter=1):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.erode(img, None, iterations=iter)

def dilate_imgs(img_list, iter=1):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.dilate(img, None, iterations=iter)

def flatten_contour(contour_list):
    tmp = contour_list[0]
    shape = tmp.shape
    aux = np.zeros(shape=(shape[0], shape[2]), dtype=int)
    for i in range(shape[0]):
        aux[i] = tmp[i].flatten()
    return aux

def find_extreme_points(contour_list):
    col_1 = contour_list[:, 0]
    col_0 = contour_list[:, 1]
    x_left, x_right = np.argmin(col_0), np.argmax(col_0)
    y_top, y_bot = np.argmax(col_1), np.argmin(col_1)
    ret = np.zeros(shape=(4, 2), dtype=int)
    ret[0, :], ret[1, :] = contour_list[x_left], contour_list[x_right]
    ret[2, :], ret[3, :] = contour_list[y_top], contour_list[y_bot]
    return ret  

def get_contour(img_list_morph, img_list, mode):
    extreme_points, i = [], 0
    for img_morph, img in zip(img_list_morph, img_list):
        cont, _ = cv2.findContours(image=img_morph, mode=mode, method=cv2.CHAIN_APPROX_NONE)
        cont_flat = flatten_contour(cont)
        extreme_points.append(find_extreme_points(cont_flat))
        i+= 1
        if i % 50 == 0: print(f'[!] {i} images processed...')
        # print(extreme_points[-1])
        img_copy = img.copy()
        # print(f'img_shape: {img_copy.shape}')
        # print(f'extreme_point: {extreme_points[-1]}')
        # for i in range(4):
        #     img_copy = cv2.circle(img_copy, extreme_points[-1][i, :], radius=5, color=(0, 255, 0), thickness=-1)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
        # cv2.drawContours(image=img_copy, contours=cont, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
    print(f'[!] Total of {i} images processed!')
    return np.array(extreme_points)  

def draw_img(img_list, index):
    if index < len(img_list):
        cv2.imshow("Brain image", img_list[index])
        cv2.waitKey()

In [4]:
INDEX = 0
HEIGHT, WIDTH = 224, 224
convert_to_int_matrix(all_images)
all_images_morph = all_images.copy()
draw_img(all_images_morph, INDEX)
imgs_to_grayscale(all_images_morph)
draw_img(all_images_morph, INDEX)  
threshold_imgs(all_images_morph)
draw_img(all_images_morph, INDEX)
erode_imgs(all_images_morph, iter=2)
draw_img(all_images_morph, INDEX)
dilate_imgs(all_images_morph, iter=4)
draw_img(all_images_morph, INDEX)
gaussian_blur(all_images_morph)
draw_img(all_images_morph, INDEX)
cv2.destroyAllWindows()
extreme_points = get_contour(all_images_morph, all_images, cv2.RETR_EXTERNAL)

(300, 240)
(1080, 1920, 3)
(664, 550, 4)
(725, 728, 4)
(512, 512, 4)
(400, 393, 4)
(400, 393, 4)
(454, 442, 4)


In [5]:
def cut_images(extreme_points, img_list):
    new_images = []
    for extreme_point, img in zip(extreme_points, img_list):
        x_left, x_right = extreme_point[0, 1], extreme_point[1, 1]
        y_top, y_bot = extreme_point[2, 0], extreme_point[3, 0]
        size_h, size_v = x_right - x_left, y_top - y_bot
        # print(f'img_shape: {img.shape}')
        # print(f'x_left: {x_left} - x_right: {x_right} - y_top: {y_top} - y_bot: {y_bot}')
        # tmp = np.zeros(shape=(size_h, size_v), dtype=np.uint8)
        # tmp = img[x_left:x_right, y_bot:y_top].copy()
        tmp = np.zeros(shape=(size_h, size_v), dtype=np.uint8)
        for i, k in zip(range(x_left, x_right), range(size_h)):
            for j, l in zip(range(y_bot, y_top), range(size_v)):
                tmp[k, l] = img[i, j]
        new_images.append(tmp)
    return new_images

In [6]:
grey_imgs = all_images.copy()
imgs_to_grayscale(grey_imgs)
new_images = cut_images(extreme_points, grey_imgs)

In [7]:
def check_images(img_morph_list, img_list):
    correct_yes_images, correct_no_images = [], []
    incorrect_yes_images, incorrect_no_images = [], []
    for img_morph, img, i in zip(img_morph_list, img_list, range(len(img_list))):
        cv2.imshow("Brain", img_morph)
        key = cv2.waitKey()
        if key == 13:
            if i < YES_SIZE:
                correct_yes_images.append(img_morph)
            else:
                correct_no_images.append(img_morph)
        elif key == 8:
            if i < YES_SIZE:
                incorrect_yes_images.append(img)
            else:
                incorrect_no_images.append(img)
        if i % 50 == 0:
            print(f'[!] {i} images checked...')
    return correct_yes_images, correct_no_images, incorrect_yes_images, incorrect_no_images

In [8]:
correct_yes_images, correct_no_images, incorrect_yes_images, incorrect_no_images = check_images(new_images, grey_imgs)
cv2.destroyAllWindows()
resize_imgs(correct_yes_images, HEIGHT, WIDTH)
resize_imgs(correct_no_images, HEIGHT, WIDTH)
# correct_yes_images = np.load('./numpy_arrays/correct_yes_images.npy', allow_pickle=True)
# correct_no_images = np.load('./numpy_arrays/correct_no_images.npy', allow_pickle=True)
# incorrect_yes_images = np.load('./numpy_arrays/incorrect_yes_images.npy', allow_pickle=True)
# incorrect_no_images = np.load('./numpy_arrays/incorrect_no_images.npy', allow_pickle=True)

[!] 0 images checked...
[!] 50 images checked...
[!] 100 images checked...
[!] 150 images checked...
[!] 200 images checked...
[!] 250 images checked...


In [9]:
print(f'[!] {len(correct_yes_images) + len(correct_no_images)} correct images!')
print(f'[!] {len(incorrect_yes_images) + len(incorrect_no_images)} incorrect images!')

[!] 203 correct images!
[!] 50 incorrect images!


In [10]:
NP_PATH = './numpy_arrays'
np.save(f'{NP_PATH}/correct_yes_images.npy', correct_yes_images, allow_pickle=True)
np.save(f'{NP_PATH}/correct_no_images.npy', correct_no_images, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_yes_images.npy', incorrect_yes_images, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_no_images.npy', incorrect_no_images, allow_pickle=True)

c:\Users\kmart\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [11]:
THRESH, INDEX = 20, 0
YES_SIZE, NO_SIZE = len(incorrect_yes_images), len(incorrect_no_images)

original_incorrect_images = np.concatenate((incorrect_yes_images, incorrect_no_images))
all_incorrect_images = original_incorrect_images.copy()

threshold_imgs(all_incorrect_images)
erode_imgs(all_incorrect_images, iter=2)
dilate_imgs(all_incorrect_images, iter=4)
gaussian_blur(all_incorrect_images)
cv2.destroyAllWindows()

extreme_points = get_contour(all_incorrect_images, original_incorrect_images, cv2.RETR_EXTERNAL)
new_images = cut_images(extreme_points, original_incorrect_images)
res_imgs = check_images(new_images, original_incorrect_images)
cv2.destroyAllWindows()

print(f'[!] {len(res_imgs[0]) + len(res_imgs[1])} correct images!')
print(f'[!] {len(res_imgs[2]) + len(res_imgs[3])} incorrect images!')

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[!] 50 images processed...
[!] Total of 50 images processed!
[!] 0 images checked...
[!] 17 correct images!
[!] 33 incorrect images!


In [12]:
resize_imgs(res_imgs[0], HEIGHT, WIDTH)
resize_imgs(res_imgs[1], HEIGHT, WIDTH)
res_correct_yes = np.concatenate((np.load(f'{NP_PATH}/correct_yes_images.npy'), np.array(res_imgs[0], dtype=np.uint8)))
res_correct_no = np.concatenate((np.load(f'{NP_PATH}/correct_no_images.npy'), np.array(res_imgs[1], dtype=np.uint8)))
print(f'[!] {len(res_correct_yes) + len(res_correct_no)} total correct images!')
print(f'[!] {TOTAL_SIZE - (len(res_correct_yes)+len(res_correct_no))} total incorrect images!')
np.save(f'{NP_PATH}/correct_yes_images_2.npy', res_correct_yes, allow_pickle=True)
np.save(f'{NP_PATH}/correct_no_images_2.npy', res_correct_no, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_yes_images_2.npy', res_imgs[2], allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_no_images_2.npy', res_imgs[3], allow_pickle=True)

[!] 220 total correct images!
[!] 33 total incorrect images!


In [13]:
class_yes = np.ones(shape=(len(res_correct_yes),), dtype=np.uint8)
class_no = np.zeros(shape=(len(res_correct_no),), dtype=np.uint8)
np.save(f'{NP_PATH}/classes.npy', np.concatenate((class_yes, class_no)), allow_pickle=True)

In [14]:
print(len(res_correct_yes))
print(len(res_correct_no))
for img in res_correct_yes:
    cv2.imshow("yes", img)
    cv2.waitKey()
cv2.destroyAllWindows()
for img in res_correct_no:
    cv2.imshow("no", img)
    cv2.waitKey()
cv2.destroyAllWindows()

134
86
